# Rough analysis of vaccination statistics by region and age

This is a rough analysis of the limited statistics on vaccination rollout published by NHS England, done as an exercise in working with these stats in the open.

It compares the total vaccinations delivered by region, against the total population and total over-80 population by region, using NHS England region population estimates published by the Office for National Statistics.

Data sources: 

- Vaccination statistics are [published by NHS England](https://www.england.nhs.uk/statistics/statistical-work-areas/covid-19-vaccinations/), as of 14 January 2021 - these are published at NHS England region level, for the under-80 and 80-plus age groups.
- Population estimates by CCG and year of age are [mid-2020 estimates from the ONS](https://www.ons.gov.uk/peoplepopulationandcommunity/populationandmigration/populationestimates/datasets/clinicalcommissioninggroupmidyearpopulationestimates) - this analysis groups these by NHS England region code, and calculates the under-80 and 80-plus populations.

Published under CC-BY-SA, please cite this repo if you use this work.

In [261]:
import csv
import pandas as pd

## Get population statistics for NHS England regions

Use local version of the relevant parts of the ONS Excel file, copied to CSV.

In [262]:
df = pd.read_csv(open("./data/population-by-ccg.csv"))
age_cols = ["All Ages"] + ([str(i) for i in list(range(0, 90))]) + ["90+"]
df[age_cols] = df[age_cols].replace(",", "", regex=True)
df[age_cols] = df[age_cols].apply(pd.to_numeric, errors='raise')
df.rename(columns={"All Ages": "total_population"}, inplace=True)
df.head()

,CCG Code,CCG Name,STP20 Code,STP20 Name,NHSER20 Code,NHSER20 Name,total_population,0,1,2,...,81,82,83,84,85,86,87,88,89,90+
0,E38000020,NHS Brent CCG,E54000027,North West London Health and Care Partnership,E40000003,London,329771,4729,4696,4955,...,1185,1201,1087,1023,866,791,681,626,508,2043
1,E38000031,NHS Central London (Westminster) CCG,E54000027,North West London Health and Care Partnership,E40000003,London,190034,1604,1798,1948,...,781,723,650,609,494,522,412,390,334,1465
2,E38000048,NHS Ealing CCG,E54000027,North West London Health and Care Partnership,E40000003,London,341806,4821,4813,4914,...,1414,1197,1188,1085,963,822,758,696,582,2236
3,E38000070,NHS Hammersmith and Fulham CCG,E54000027,North West London Health and Care Partnership,E40000003,London,185143,2214,2241,2297,...,582,528,473,419,394,340,359,281,268,953
4,E38000074,NHS Harrow CCG,E54000027,North West London Health and Care Partnership,E40000003,London,251160,3545,3606,3587,...,1234,1128,1059,1064,968,848,702,627,581,2280


Get proportion in each age group, by region, and eyeball the results to make sure they look legit.

In [281]:
df_grouped = df.groupby(["NHSER20 Code", "NHSER20 Name"]).sum()
ages_below_80 = [str(i) for i in list(range(0, 80))]
ages_80_plus = [str(i) for i in list(range(80, 90))] + ["90+"]
df_grouped['total_population_below_80'] = df_grouped[ages_below_80].sum(axis=1)
df_grouped['total_population_80_plus'] = df_grouped[ages_80_plus].sum(axis=1)
df_grouped['proportion_population_below_80'] = df_grouped.total_population_below_80 / \
    df_grouped.total_population * 100
df_grouped['proportion_population_80_plus'] = df_grouped.total_population_80_plus / \
    df_grouped.total_population * 100
df_grouped.reset_index(inplace=True)
print(sum(df_grouped.total_population))
df_grouped

56286961


,NHSER20 Code,NHSER20 Name,total_population,0,1,2,3,4,5,6,...,85,86,87,88,89,90+,total_population_below_80,total_population_80_plus,proportion_population_below_80,proportion_population_80_plus
0,E40000003,London,8961989,117840,120779,122349,123655,121381,120529,123395,...,22756,20672,18726,16572,14735,57456,8660256,301733,96.633192,3.366808
1,E40000005,South East,8897835,92296,96962,100664,105041,106267,107051,109805,...,37452,34233,31578,28569,24268,97679,8398580,499255,94.389028,5.610972
2,E40000006,South West,5631114,53635,56319,58355,61444,61645,63022,64562,...,26089,24146,21891,19623,17106,66232,5284080,347034,93.837205,6.162795
3,E40000007,East of England,6529013,71711,75606,78066,81246,81143,81980,83782,...,27321,24403,22721,20159,17646,66639,6170961,358052,94.515986,5.484014
4,E40000008,Midlands,10601877,114847,119981,124465,128281,128096,130029,133200,...,40633,36811,33854,29676,25575,95366,10061478,540399,94.902799,5.097201
5,E40000009,North East and Yorkshire,8603721,89727,93577,97734,100089,100216,101460,103967,...,34269,30512,27404,24281,20498,74348,8159306,444415,94.834619,5.165381
6,E40000010,North West,7061412,78802,80832,83963,86379,86244,87051,88031,...,26207,23230,21225,18890,16047,59553,6715336,346076,95.099054,4.900946


Now calculate what proportion of the overall and 80+ population lives in each region.

In [279]:
total_england_popululation = df_grouped["total_population"].sum()
total_england_population_80_plus = df_grouped["total_population_80_plus"].sum()
df_grouped['regions_proportion_of_total_england_population'] = df_grouped.total_population / \
    total_england_popululation * 100
df_grouped['regions_proportion_of_80_plus_england_population'] = df_grouped.total_population_80_plus / \
    total_england_population_80_plus * 100
df_grouped['percent_of_population_over_80'] = df_grouped.total_population_80_plus / \
    df_grouped.total_population * 100

Check the bits we care about.

In [280]:
cols = ["NHSER20 Code", "NHSER20 Name", "total_population", 
        "total_population_below_80", 
        "total_population_80_plus", 
        "regions_proportion_of_total_england_population", 
        "regions_proportion_of_80_plus_england_population",
        "percent_of_population_over_80"]
df_grouped[cols]

,NHSER20 Code,NHSER20 Name,total_population,total_population_below_80,total_population_80_plus,regions_proportion_of_total_england_population,regions_proportion_of_80_plus_england_population,percent_of_population_over_80
0,E40000003,London,8961989,8660256,301733,15.921963,10.635771,3.366808
1,E40000005,South East,8897835,8398580,499255,15.807986,17.598214,5.610972
2,E40000006,South West,5631114,5284080,347034,10.004296,12.232584,6.162795
3,E40000007,East of England,6529013,6170961,358052,11.599512,12.620957,5.484014
4,E40000008,Midlands,10601877,10061478,540399,18.835405,19.048497,5.097201
5,E40000009,North East and Yorkshire,8603721,8159306,444415,15.285460,15.665162,5.165381
6,E40000010,North West,7061412,6715336,346076,12.545378,12.198815,4.900946


In [266]:
df_grouped.to_csv("./data/region-populations.csv", index=False)

## Compare total population against vaccinations delivered

Now look at the proportion of vaccinations delivered to each region, and compare that against the population figures.

In [267]:
df_vacc = pd.read_csv(open("./data/vaccinations-14-jan.csv"))
df_vacc.rename(columns={"Total doses to date": "total_doses"}, inplace=True)
df_vacc = df_vacc[~df_vacc["Region of residence"].isnull()]

In [268]:
total_doses = sum(df_vacc.total_doses)
print(total_vaccs) # good, this is in line with the total reported elsewhere.
df_vacc['percent_of_total_england_doses'] = df_vacc.total_doses / total_doses * 100
df_vacc['total_first_doses'] = df_vacc['1st dose - under 80'] + df_vacc["1st dose - 80+"]
df_vacc['percent_of_first_doses_to_over_80s'] = df_vacc['1st dose - 80+'] / df_vacc.total_first_doses
# df_vacc[''] = df_vacc['1st dose - under 80'] / df
df_vacc

2371407


,Region of residence,1st dose - under 80,1st dose - 80+,2nd dose - under 80,2nd dose - 80+,total_doses,NHSER code,percent_of_total_england_doses,total_first_doses,percent_of_first_doses_to_over_80s
0,Other,1371,759,162,160,2452,NaN,0.103399,2130,0.356338
1,London,107588,92398,9198,28340,237524,E40000003,10.016163,199986,0.462022
2,North West,135178,131407,14146,37714,318445,E40000010,13.428526,266585,0.492927
3,Midlands,194628,193019,14803,44879,447329,E40000008,18.863443,387647,0.497925
4,South East,165564,183299,14147,48247,411257,E40000005,17.342320,348863,0.525418
5,South West,108212,126896,9655,40569,285332,E40000006,12.032182,235108,0.539735
6,North East And Yorkshire,166554,204140,12226,50125,433045,E40000009,18.261100,370694,0.550697
7,East Of England,81604,104687,6891,42841,236023,E40000007,9.952868,186291,0.561954


Merge with the population stats, and add more indicators.

In [276]:
df_merged = pd.merge(df_vacc[df_vacc['Region of residence'] != "Other"], df_grouped[cols], left_on='NHSER code', right_on='NHSER20 Code', how='left')
df_merged['vaccinations_per_head'] = df_merged.total_doses / df_merged.total_population
df_merged['percent_first_doses_given_to_over_80s'] = \
    df_merged['1st dose - 80+'] / (df_merged['1st dose - 80+'] + df_merged['2nd dose - 80+'] ) * 100
df_merged['percent_over_80s_vaccinated'] = \
    df_merged['1st dose - 80+'] / df_merged.total_population_80_plus * 100

In [277]:
all_cols = cols + ["percent_of_total_england_doses", "vaccinations_per_head",
                   "percent_first_doses_given_to_over_80s",
                   "percent_over_80s_vaccinated"]
df_merged[all_cols].sort_values("percent_of_total_england_doses", ascending=False)

,NHSER20 Code,NHSER20 Name,total_population,total_population_below_80,total_population_80_plus,regions_proportion_of_total_england_population,regions_proportion_of_80_plus_england_population,percent_of_total_england_doses,vaccinations_per_head,percent_first_doses_given_to_over_80s,percent_over_80s_vaccinated
2,E40000008,Midlands,10601877,10061478,540399,18.835405,19.048497,18.863443,0.042193,81.135192,35.717868
5,E40000009,North East and Yorkshire,8603721,8159306,444415,15.285460,15.665162,18.261100,0.050332,80.286315,45.934543
3,E40000005,South East,8897835,8398580,499255,15.807986,17.598214,17.342320,0.046220,79.163104,36.714505
1,E40000010,North West,7061412,6715336,346076,12.545378,12.198815,13.428526,0.045097,77.699990,37.970561
4,E40000006,South West,5631114,5284080,347034,10.004296,12.232584,12.032182,0.050671,75.774639,36.565870
0,E40000003,London,8961989,8660256,301733,15.921963,10.635771,10.016163,0.026503,76.527688,30.622438
6,E40000007,East of England,6529013,6170961,358052,11.599512,12.620957,9.952868,0.036150,70.960767,29.237932


In [ ]:
df_merged[all_cols].sort_values("percent_of_total_england_doses", ascending=False)\
    .to_csv("./data/summary-for-chart.csv", index=False)